In [1]:
from bs4 import BeautifulSoup
import requests
import selenium
import random
from time import sleep
import pandas as pd
import logging 
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='hday_session.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)
from datetime import datetime
import pickle

In [2]:
pd.set_option('display.max_rows', 1000)

#  Constants

In [3]:
baseurl = 'https://www.tripadvisor.com.ph'

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0'} 

In [5]:
def urlToBs(url, headers):
    sleep(random.uniform(0,0.5))
    pg = requests.get(url, headers)
    sleep(1)
    bs = BeautifulSoup(pg.content, 'html.parser')
    return bs

## Workflow
1. loop from 1- 600
    get urls 
2. loop on urls
    get individual data

##  Pages Urls

In [6]:
def sp_to_lturl(bsPage):
    return [a.find("a")['href'] for a in bsPage.find_all("div", {'class':"listing_title"})] 

In [7]:
pageUrls = []
for i in range(21):
    if i  == 0:
        url = baseurl + '/Hotels-g294260-Boracay_Malay_Aklan_Province_Panay_Island_Visayas-Hotels.html#BODYCON'
    else:
        url = baseurl +  '/Hotels-g294260-oa' + str(i*30) + '-Boracay_Malay_Aklan_Province_Panay_Island_Visayas-Hotels.html#BODYCON'
    pageUrls.append(url)

In [8]:
pageUrls[0]

'https://www.tripadvisor.com.ph/Hotels-g294260-Boracay_Malay_Aklan_Province_Panay_Island_Visayas-Hotels.html#BODYCON'

In [9]:
hturllt = []
for purls in pageUrls:
    logger.info(purls)
    bsPage = urlToBs(purls, headers)
    hturllt += sp_to_lturl(bsPage) 

In [10]:
len(hturllt)

586

In [11]:
hotelUrls =  [baseurl+str(url) for url in hturllt]

##  Hotel Details 

1. ratings, ok
2. number of amenities offered by hotels - ok
3. number of reviews per hotel - ok
4. hotel price - ok
5. feedback from guests - another loop (eto ba yung text)
6. hotel class (3-star, 4-star, 5-star) - ok
7. room features -  ok 
8. number of nearby restaurants -ok
9. number of nearby attractions - ok
10. hotel location - ??

###  Scratch pad

In [11]:
def bsclass_text(bs,clstr):
    return bs.find(class_ = clstr).get_text() 

In [12]:
def getprice(bsHotel):
    if bsHotel.find(class_ = 'hotels-hotel-offers-DetailChevronOffer__price--py2LH'):
        price = bsHotel.find(
            class_ = 'hotels-hotel-offers-DetailChevronOffer__price--py2LH').get_text()
    elif bsHotel.find(class_ = 'hotels-hotel-offers-DominantOffer__price--D-ycN'):
        price = bsHotel.find(
            class_ = 'hotels-hotel-offers-DominantOffer__price--D-ycN').get_text()
    else:
        price = ''
    return price 

In [13]:
def getAllReviews(bsHotel):
    ltord = [a.get_text() for a in bsHotel.find_all(class_ = 'location-review-review-list-parts-ReviewRatingFilter__row_num--3cSP7')]
    if len(ltord) == 0:
        ltord = ['','','','','']
    return tuple(ltord)

In [14]:
def getHotelclass(bsHotel):
    if bsHotel.find(class_ = '_2MgVjxWG'):
        hClass = 2
    elif bsHotel.find(class_ = '_3RprXHxE'):
        hClass = 3
    elif bsHotel.find(class_ = '_30WZSV_9'):
        hClass = 4
    elif bsHotel.find(class_ = '_2LYcDtDf'):
        hClass = 4.5
    elif bsHotel.find(class_ = 'f33bWmtw'):
        hClass = 5 
    else:
        hClass = 0
    return hClass

In [15]:
def getNearby(bsHotel):
    if bsHotel.find(class_ = 'hotels-hotel-review-location-layout-Highlight__orange--1N-BP'):
        nearbyResto = bsHotel.find(
        class_ = 'hotels-hotel-review-location-layout-Highlight__orange--1N-BP').get_text()
    else:
        nearbyResto  = ''
    if bsHotel.find(class_ = 'hotels-hotel-review-location-layout-Highlight__blue--2qc3K'):
        nearbyAttractions = bsHotel.find(
        class_ = 'hotels-hotel-review-location-layout-Highlight__blue--2qc3K').get_text()
    else:
        nearbyAttractions = ''
    return  nearbyResto, nearbyAttractions

In [16]:
def getRating(bsHotel):
    if bsHotel.find(class_ = 'hotels-hotel-review-about-with-photos-Reviews__overallRating--vElGA'):
        rating = bsHotel.find(
        class_ = 'hotels-hotel-review-about-with-photos-Reviews__overallRating--vElGA').get_text()
    else:
        rating  = ''
    return rating

In [17]:
def getTotalRev(bsHotel):
    if bsHotel.find(class_ = 'hotels-community-content-common-TabAboveHeader__tabCount--26Tct'):
        totalRev = bsHotel.find(
        class_ = 'hotels-community-content-common-TabAboveHeader__tabCount--26Tct').get_text()
    else:
        totalRev  = ''
    return totalRev

In [18]:
def getHotelDetails(bsHotel):
    title =  bsHotel.find(id='HEADING').get_text()
    price = getprice(bsHotel)
    hotelClass = getHotelclass(bsHotel)
    rating = getRating(bsHotel)
    totalRev = getTotalRev(bsHotel)
    excRev, vgRev, aveRev, poorRev, terRev =  getAllReviews(bsHotel)
    nearbyResto, nearbyAttractions = getNearby(bsHotel)
        
    return {'title': title
            , 'price': price
            , 'hotelClass': hotelClass
            , 'rating' : rating
            , 'totalRev': totalRev
            , 'excellentRev':excRev
            , 'verygoodRev':vgRev
            , 'averageRev':aveRev
            , 'poorRev':poorRev
            , 'terribleRev':terRev
            , 'nearbyRestaurant':nearbyResto
            , 'nearbyAttractions':nearbyAttractions
           }

## Individual Data looper

In [19]:
detailslt  = []
i = 0
for hurl in hotelUrls:
    i += 1
    logger.info(i)
    logger.info(hurl)
    bsHotel = urlToBs(hurl, headers)
    detailslt.append(getHotelDetails(bsHotel))

In [20]:
i = 322
for hurl in hotelUrls[322:]:
    i += 1
    logger.info(i)
    logger.info(hurl)
    bsHotel = urlToBs(hurl, headers)
    detailslt.append(getHotelDetails(bsHotel))

In [21]:
len(detailslt)

902

In [22]:
detailslt[-1]

{'title': "Evelyn's Place",
 'price': '',
 'hotelClass': 2,
 'rating': '',
 'totalRev': '',
 'excellentRev': '',
 'verygoodRev': '',
 'averageRev': '',
 'poorRev': '',
 'terribleRev': '',
 'nearbyRestaurant': '16',
 'nearbyAttractions': '6'}

In [23]:
hotelUrls[322]

'https://www.tripadvisor.com.ph/Hotel_Review-g294260-d1201916-Reviews-Ocean_Breeze_Inn_Boracay-Boracay_Malay_Aklan_Province_Panay_Island_Visayas.html'

In [24]:
len(detailslt)

902

In [25]:
bsdf  = pd.DataFrame(detailslt)

In [43]:
bsdf.head(600)

,title,price,hotelClass,rating,totalRev,excellentRev,verygoodRev,averageRev,poorRev,terribleRev,nearbyRestaurant,nearbyAttractions
0,Calypso Beach & Dive Resort,"₱2,711",3.0,4.5,107,66,24,8,5,4,50,5
1,Henann Palm Beach Resort,"₱5,893",3.0,4.5,273,224,26,10,5,8,53,5
2,Henann Lagoon Resort,"₱3,395",4.5,4.5,832,516,231,55,12,18,47,6
3,Fridays Boracay,"₱18,810",4.0,4.0,672,349,195,58,39,31,56,10
4,Two Seasons Boracay Resort,"₱5,666",4.0,4.0,614,280,234,71,9,20,24,5
...,...,...,...,...,...,...,...,...,...,...,...,...
595,Blue Pirates Resort by Boracay Travellers',,0.0,,,,,,,,276,43
596,Calypso Beach Front Resort,,3.0,,,,,,,,276,43
597,Gumamela Resort,,3.0,,,,,,,,16,6
598,Kimberly Suites,,3.0,,,,,,,,28,4


In [27]:
tsbsdf = datetime.now().strftime('%H_%M_%S')

In [28]:
tsbsdf

'12_53_44'

In [29]:
bsdf.to_csv(f'bsdf_{tsbsdf}.csv')

In [36]:
len(bsdf)

902

In [38]:
bsdf

,title,price,hotelClass,rating,totalRev,excellentRev,verygoodRev,averageRev,poorRev,terribleRev,nearbyRestaurant,nearbyAttractions
0,Calypso Beach & Dive Resort,"₱2,711",3.0,4.5,107,66,24,8,5,4,50,5
1,Henann Palm Beach Resort,"₱5,893",3.0,4.5,273,224,26,10,5,8,53,5
2,Henann Lagoon Resort,"₱3,395",4.5,4.5,832,516,231,55,12,18,47,6
3,Fridays Boracay,"₱18,810",4.0,4.0,672,349,195,58,39,31,56,10
4,Two Seasons Boracay Resort,"₱5,666",4.0,4.0,614,280,234,71,9,20,24,5
...,...,...,...,...,...,...,...,...,...,...,...,...
897,Hotel Mari Annex,,2.0,,,,,,,,115,6
898,Wald's Place,,2.0,,,,,,,,36,3
899,El Dominador Guest House,,2.0,,,,,,,,,
900,Cliff House,,2.0,,,,,,,,,


##  Some parts are note accessible via beautiful soup, selenium is a must

In [12]:
from selenium import webdriver
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options 

In [13]:
opts = Options()
opts.headless = True
brow = Firefox(options = opts)

In [14]:
def getAmenitiesandFeatures(selbrow):
    try:
        #click show more
        selbrow.find_elements_by_class_name(
            'hotels-hr-about-amenities-AmenityGroup__showMore--pPz2S')[0].click()
        sleep(random.uniform(0,0.5))
        #click property amenities
        selbrow.find_elements_by_class_name(
            'hotels-hr-about-amenities-AmenitiesModal__tabContainer--F67_I')[0].click()
        sleep(random.uniform(0,0.5))
        ltAmen =  [i.text for i in selbrow.find_elements_by_class_name('hotels-hr-about-amenities-Amenity__amenity--3fbBj')
                  if i.text != '']
        selbrow.find_elements_by_class_name(
        'hotels-hr-about-amenities-AmenitiesModal__tabContainer--F67_I')[1].click()
        sleep(random.uniform(0,0.5))
        ltfeatures =  [i.text for i in selbrow.find_elements_by_class_name('hotels-hr-about-amenities-Amenity__amenity--3fbBj')
                      if i.text != '']
        selbrow.find_elements_by_class_name('_2EFRp_bb')[0].click()
        ltAmen = list(set(ltAmen))
        ltfeatures = list(set(ltfeatures))
    except:
        logger.error("Selenium Error")
        ltAmen = []
        ltfeatures = []
    return ltAmen, ltfeatures  

In [15]:
def getDetailsfromSel(selbrow):
    title =  selbrow.find_element_by_id('HEADING').text
    amenitieslt, roomfeatureslt  = getAmenitiesandFeatures(selbrow)
    return {'title': title
            , 'amenities': amenitieslt
            , 'roomfeatures': roomfeatureslt
           }
        

In [ ]:
selhoteldetslt = []
i = 0
for url in hotelUrls:
    i += 1
    logger.info(i)
    logger.info(url)
    sleep(random.uniform(0,0.5))
    brow.get(url)
    sleep(1)
    selhoteldetslt.append(getDetailsfromSel(brow))
    

In [ ]:
i = 56
for url in hotelUrls[56:]:
    i += 1
    logger.info(i)
    logger.info(url)
    sleep(random.uniform(3,4))
    brow.get(url)
    selhoteldetslt.append(getDetailsfromSel(brow))
    

In [ ]:
len(selhoteldetslt)

In [ ]:
seldf = pd.DataFrame(selhoteldetslt)

In [ ]:
df = bsdf.merge(seldf, on = 'title')

In [ ]:
tsdf = datetime.now().strftime('%H_%M_%S')

In [ ]:
df.to_csv(f'df_{tsdf}.csv')